In [1]:
import os
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import ECB, CBC, CTR
from PIL import Image
import numpy as np
import time
from tqdm import tqdm
import copy
from random import randrange, randint
from IPython.display import Image as show_image

# Przykład

In [2]:
plaintext = b'Fundamental Cryptography in Python'
print(f"Plaintext ({len(plaintext)}): {plaintext}")

Plaintext (34): b'Fundamental Cryptography in Python'


In [3]:
# 256-bit symmetric key
key = os.urandom(32)
print(key)
print(len(key))

b'\x9b\r6\x10"$8g\xe4\xc9~\x9dL\xe0\xa7\x83\x9e\xee\x18;3\x91\xe1\xb9\xfd\xa9\x10\xc0\x18\x81\x99G'
32


In [4]:
# AES ECB cipher
aes_ecb_cipher = Cipher(AES(key), ECB())

In [5]:
# Encrypt without padding
ciphertext = aes_ecb_cipher.encryptor().update(plaintext)
print(f"Ciphertext (no padding) ({len(ciphertext)}): {ciphertext}")

Ciphertext (no padding) (32): b'kBs\xd2\x94\x18\x93\x9a|\x86\x1cSZ\x82\xca<\xb6\x9cG\x17\xbf\xb8\xc3\x91a\xe3\xa3\x93\xc8\x01R\xe8'


In [6]:
# Decrypt without padding
recovered_plaintext = aes_ecb_cipher.decryptor().update(ciphertext)
print(f"Recovered plaintext (no padding) ({len(recovered_plaintext)}): {recovered_plaintext}")

Recovered plaintext (no padding) (32): b'Fundamental Cryptography in Pyth'


In [7]:
# Pad the plaintext
aes_block_size_in_bits = 128
pkcs7_padder = padding.PKCS7(aes_block_size_in_bits).padder()
padded_plaintext = pkcs7_padder.update(plaintext) + pkcs7_padder.finalize()
print(f"Padded plaintext: {padded_plaintext}")

Padded plaintext: b'Fundamental Cryptography in Python\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e'


In [8]:
# Encrypt with padding
ciphertext = aes_ecb_cipher.encryptor().update(padded_plaintext)
print(f"Ciphertext ({len(ciphertext)}): {ciphertext}")

Ciphertext (48): b'kBs\xd2\x94\x18\x93\x9a|\x86\x1cSZ\x82\xca<\xb6\x9cG\x17\xbf\xb8\xc3\x91a\xe3\xa3\x93\xc8\x01R\xe8,\x82\xe0\x19\xee\n\x8c\xc9\x93\xc9\x19\xe6\xd5\x8c\xd3\xed'


In [9]:
# Decrypt with padding
recovered_plaintext_with_padding = aes_ecb_cipher.decryptor().update(ciphertext)
print(f"Recovered plaintext + padding: {recovered_plaintext_with_padding}")

Recovered plaintext + padding: b'Fundamental Cryptography in Python\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e'


In [10]:
pkcs7_unpadder = padding.PKCS7(aes_block_size_in_bits).unpadder()
recovered_plaintext = pkcs7_unpadder.update(recovered_plaintext_with_padding) + pkcs7_unpadder.finalize()
print(f"Recovered plaintext: {recovered_plaintext}")
assert (plaintext == recovered_plaintext)

Recovered plaintext: b'Fundamental Cryptography in Python'


# Zadania

In [11]:
img_data = np.array(Image.open('data/1mb.png').convert('RGBA'))
im1mb = img_data.tobytes()
im5mb = np.array(Image.open('data/5mb.png').convert('RGBA')).tobytes()
im10mb = np.array(Image.open('data/10mb.png').convert('RGBA')).tobytes()

E:\anaconda3\envs\krypto\lib\site-packages\PIL\Image.py:3406: DecompressionBombWarning: Image size (170664000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [12]:
im1mb[0:100]

b'\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00\xff\xff\xff\x00'

In [13]:
aes_block_size_in_bits = 128
aes_block_size_in_bytes = aes_block_size_in_bits // 8
key = os.urandom(32)
print(key)
print(len(key))

b'Y\x0f\x03\x9eW\x92\x92\x91\x9fT\x1aB.YR\x1ej\xf5\xeey\x84\xf8\xe5D\xe9\x00=0\xebD/\xc4'
32


In [14]:
pkcs7_padder = padding.PKCS7(aes_block_size_in_bits).padder()
im1mb_padded = pkcs7_padder.update(im1mb) + pkcs7_padder.finalize()
pkcs7_padder = padding.PKCS7(aes_block_size_in_bits).padder()
im5mb_padded = pkcs7_padder.update(im5mb) + pkcs7_padder.finalize()
pkcs7_padder = padding.PKCS7(aes_block_size_in_bits).padder()
im10mb_padded = pkcs7_padder.update(im10mb) + pkcs7_padder.finalize()

In [15]:
if not os.path.isdir('results'):
    os.mkdir('results')

In [16]:
num_of_measurements = 50

In [17]:
def save_img(path, image_array):
    img = Image.fromarray(image_array, 'RGBA')
    img.save(path)

In [18]:
def measure_enc_time(enc_method):
    total_1mb_enc_times = np.zeros(num_of_measurements)
    total_5mb_enc_times = np.zeros(num_of_measurements)
    total_10mb_enc_times = np.zeros(num_of_measurements)
    
    for i in tqdm(range(num_of_measurements)):
        start = time.time()
        cipherdata_1mb = enc_method(im1mb_padded)
        stop = time.time()
        total_1mb_enc_times[i] = stop - start
        
        start = time.time()
        cipherdata_5mb = enc_method(im5mb_padded)
        stop = time.time()
        total_5mb_enc_times[i] = stop - start
        
        start = time.time()
        cipherdata_10mb = enc_method(im10mb_padded)
        stop = time.time()
        total_10mb_enc_times[i] = stop - start
    
    print(f"For 1mb.png, mean = {total_1mb_enc_times.mean():.4f}s, var = {total_1mb_enc_times.var():.4f}s, median = {np.median(total_1mb_enc_times):.4f}s")
    print(f"For 5mb.png, mean = {total_5mb_enc_times.mean():.4f}s, var = {total_5mb_enc_times.var():.4f}s, median = {np.median(total_5mb_enc_times):.4f}s")
    print(f"For 10mb.png, mean = {total_10mb_enc_times.mean():.4f}s, var = {total_10mb_enc_times.var():.4f}s, median = {np.median(total_10mb_enc_times):.4f}s")
    return cipherdata_1mb, cipherdata_5mb, cipherdata_10mb

In [19]:
def measure_dec_time(dec_method, cipherdata_1mb, cipherdata_5mb, cipherdata_10mb):
    # mierzę czysty czas deszyfrowania bez usuwania paddingu
    total_1mb_dec_times = np.zeros(num_of_measurements)
    total_5mb_dec_times = np.zeros(num_of_measurements)
    total_10mb_dec_times = np.zeros(num_of_measurements)
    
    for i in tqdm(range(num_of_measurements)):
        start = time.time()
        image_data_1mb = dec_method(cipherdata_1mb)
        stop = time.time()
        total_1mb_dec_times[i] = stop - start
        
        start = time.time()
        image_data_5mb = dec_method(cipherdata_5mb)
        stop = time.time()
        total_5mb_dec_times[i] = stop - start
        
        start = time.time()
        image_data_10mb = dec_method(cipherdata_10mb)
        stop = time.time()
        total_10mb_dec_times[i] = stop - start
    
    assert(image_data_1mb == im1mb_padded)
    assert(image_data_5mb == im5mb_padded)
    assert(image_data_10mb == im10mb_padded)
    
    print(f"For 1mb.png, mean = {total_1mb_dec_times.mean():.4f}s, var = {total_1mb_dec_times.var():.4f}s, median = {np.median(total_1mb_dec_times):.4f}s")
    print(f"For 5mb.png, mean = {total_5mb_dec_times.mean():.4f}s, var = {total_5mb_dec_times.var():.4f}s, median = {np.median(total_5mb_dec_times):.4f}s")
    print(f"For 10mb.png, mean = {total_10mb_dec_times.mean():.4f}s, var = {total_10mb_dec_times.var():.4f}s, median = {np.median(total_10mb_dec_times):.4f}s")

In [20]:
def decrypt_image(image_cipherdata, decrypt_method):
    decrypted_img_padded = decrypt_method(image_cipherdata)
    pkcs7_unpadder = padding.PKCS7(aes_block_size_in_bits).unpadder()
    decrypted_img = pkcs7_unpadder.update(decrypted_img_padded) + pkcs7_unpadder.finalize()
    return np.frombuffer(decrypted_img, dtype=np.uint8)[:img_data.size].reshape(img_data.shape)

In [21]:
def corrupt_cipherdata(cipherdata, num_of_changes=1):
    cipherdata_corrupted = bytearray(copy.copy(cipherdata))
    assert cipherdata_corrupted == cipherdata
    old_size = len(cipherdata_corrupted)
    for i in tqdm(range(num_of_changes)):
        random_pos = randrange(len(cipherdata_corrupted) // 2)
        cipherdata_corrupted[random_pos] = cipherdata_corrupted[random_pos+1]
    new_size = len(cipherdata_corrupted)
    assert old_size == new_size
    assert cipherdata_corrupted != cipherdata
    return cipherdata_corrupted

## ECB

In [22]:
if not os.path.isdir('results/ecb'):
    os.mkdir('results/ecb')

In [23]:
# AES ECB cipher
aes_ecb_cipher = Cipher(AES(key), ECB())

In [24]:
# Encrypt with padding
cipherdata_1mb = aes_ecb_cipher.encryptor().update(im1mb_padded)

In [25]:
# Encrypt with padding
cipherdata_5mb = aes_ecb_cipher.encryptor().update(im5mb_padded)

In [26]:
# Encrypt with padding
cipherdata_10mb = aes_ecb_cipher.encryptor().update(im10mb_padded)

In [27]:
len(cipherdata_1mb)

34560016

In [28]:
encrypted_img_data = np.frombuffer(cipherdata_1mb, dtype=np.uint8)[:img_data.size].reshape(img_data.shape)

In [29]:
encrypted_img = Image.fromarray(encrypted_img_data, 'RGBA')
encrypted_img.save('results/ecb/1mb.png')

In [ ]:
show_image(filename='results/ecb/1mb.png', width=200)

### Pomiar czasu szyfrowania ECB

In [30]:
cipherdata_1mb, cipherdata_5mb, cipherdata_10mb = measure_enc_time(lambda data: aes_ecb_cipher.encryptor().update(data))

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:22<00:00,  2.19it/s]

For 1mb.png, mean = 0.0167s, var = 0.0000s, median = 0.0160s
For 5mb.png, mean = 0.1256s, var = 0.0000s, median = 0.1250s
For 10mb.png, mean = 0.3136s, var = 0.0001s, median = 0.3130s


### Pomiar czasu deszyfrowania ECB

In [31]:
measure_dec_time(lambda data: aes_ecb_cipher.decryptor().update(data), cipherdata_1mb, cipherdata_5mb, cipherdata_10mb)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:22<00:00,  2.19it/s]

For 1mb.png, mean = 0.0166s, var = 0.0000s, median = 0.0160s
For 5mb.png, mean = 0.1251s, var = 0.0000s, median = 0.1250s
For 10mb.png, mean = 0.3139s, var = 0.0000s, median = 0.3133s


In [32]:
if not os.path.isdir('results/error_prop'):
    os.mkdir('results/error_prop')
if not os.path.isdir('results/error_prop/ecb'):
    os.mkdir('results/error_prop/ecb')

In [33]:
decrypted_1b = decrypt_image(cipherdata_1mb, lambda data: aes_ecb_cipher.decryptor().update(data))

In [34]:
save_img('results/error_prop/ecb/1mb.png', decrypted_1b)

### Błąd w szyfrogramie

In [35]:
cipherdata_1mb_corrupted = corrupt_cipherdata(cipherdata_1mb)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [36]:
decrypted_1b_corrupted = decrypt_image(cipherdata_1mb_corrupted, lambda data: aes_ecb_cipher.decryptor().update(data))
save_img('results/error_prop/ecb/1mb_corrupted.png', decrypted_1b_corrupted)

In [ ]:
show_image(filename='results/error_prop/ecb/1mb_corrupted.png', width=200)

## CBC

In [38]:
iv = os.urandom(aes_block_size_in_bytes)
aes_cbc_cipher = Cipher(AES(key), CBC(iv))

In [39]:
msg = b'To jest testowa wiadomosc. Dalszy teskt. ABCDEFGHIJK'
pkcs7_unpadder = padding.PKCS7(aes_block_size_in_bits).padder()
padded_msg = pkcs7_unpadder.update(msg) + pkcs7_unpadder.finalize()
encrypted_msg = aes_cbc_cipher.encryptor().update(padded_msg)
print(encrypted_msg)

b'K6\xd1\x12\xb6\xf4\x0e[\xb1\xec(\x06\x81\x9c\xa3\xb8\xd2\xf6\x18\xcd\x05\xaf\x7f\x97\xba\x0c\xb1\xf5,N\x01Oqu|\x81\x96h\xd8Q\xe9\xd5?P\xfcF*KFX\xac\x8c6\xbd\xb0m\x9cWORS\xd4"\xa0'


In [40]:
decrypted_msg = aes_cbc_cipher.decryptor().update(encrypted_msg)
print(decrypted_msg)

b'To jest testowa wiadomosc. Dalszy teskt. ABCDEFGHIJK\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c'


### Pomiar czasu szyfrowania CBC

In [41]:
cipherdata_1mb, cipherdata_5mb, cipherdata_10mb = measure_enc_time(lambda data: aes_cbc_cipher.encryptor().update(data))

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:50<00:00,  1.02s/it]

For 1mb.png, mean = 0.0372s, var = 0.0000s, median = 0.0360s
For 5mb.png, mean = 0.2827s, var = 0.0003s, median = 0.2735s
For 10mb.png, mean = 0.6966s, var = 0.0015s, median = 0.6733s


### Pomiar czasu deszyfrowania CBC

In [42]:
measure_dec_time(lambda data: aes_cbc_cipher.decryptor().update(data), cipherdata_1mb, cipherdata_5mb, cipherdata_10mb)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.17it/s]

For 1mb.png, mean = 0.0169s, var = 0.0000s, median = 0.0160s
For 5mb.png, mean = 0.1269s, var = 0.0001s, median = 0.1250s
For 10mb.png, mean = 0.3168s, var = 0.0002s, median = 0.3130s


### Błąd w szyfrogramie tekstowym

In [93]:
msg = b'To jest testowa wiadomosc. Powinna byc podzielona na wiele blokow. Dalsza czesc wiadomosci. ABCDEFGHIJK'
pkcs7_unpadder = padding.PKCS7(aes_block_size_in_bits).padder()
padded_msg = pkcs7_unpadder.update(msg) + pkcs7_unpadder.finalize()
print(padded_msg, len(padded_msg))

b'To jest testowa wiadomosc. Powinna byc podzielona na wiele blokow. Dalsza czesc wiadomosci. ABCDEFGHIJK\t\t\t\t\t\t\t\t\t' 112


In [94]:
num_of_blocks = len(padded_msg) // aes_block_size_in_bytes
num_of_blocks

7

In [105]:
margin = 1
from_where_corrupt = randint(margin, num_of_blocks - margin)
corrupt_pos = from_where_corrupt * aes_block_size_in_bytes
print(from_where_corrupt)
print(corrupt_pos)

3
48


In [106]:
first_half = aes_cbc_cipher.encryptor().update(padded_msg[0:corrupt_pos])
print(first_half, len(first_half))

b'K6\xd1\x12\xb6\xf4\x0e[\xb1\xec(\x06\x81\x9c\xa3\xb8\x0e\xbbe\xaf\xf5\xa1W\xb57%\xbae\xef\xd5\xf2\x00\x97\x07\x9bg&\x18\xed<\xcb\xd9\xb6\xf1\xf4\x9b\x19\xce' 48


In [107]:
second_half_iv = bytes(corrupt_cipherdata(first_half[-aes_block_size_in_bytes:], 2))
print(second_half_iv, len(second_half_iv))
assert second_half_iv != first_half[-aes_block_size_in_bytes:]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]

b'\x97\x07gg&\x18\xed<\xcb\xd9\xb6\xf1\xf4\x9b\x19\xce' 16


In [108]:
aes_cbc_cipher2 = Cipher(AES(key), CBC(os.urandom(16)))
second_half = aes_cbc_cipher2.encryptor().update(padded_msg[corrupt_pos:])
print(second_half, len(second_half))

b'\xbc\xf0\x02\x8a\xbc\xaf\xc2$\xfbZm\x89\xf7\xc1\xa9\xc0@\xfb?\x97\x16\xae I\xc6\x98\xab\x87\x95-\xde\xfd2&\xe2\xf8\x89.\x1b_\xa4:\xf68\x05\x18_\xadM\xd5\x85\xd0a_SW\xf18\xc9\xe8\xe2B!\x8e' 64


In [109]:
corrupted_cipherdata = first_half + second_half
print(corrupted_cipherdata)

b'K6\xd1\x12\xb6\xf4\x0e[\xb1\xec(\x06\x81\x9c\xa3\xb8\x0e\xbbe\xaf\xf5\xa1W\xb57%\xbae\xef\xd5\xf2\x00\x97\x07\x9bg&\x18\xed<\xcb\xd9\xb6\xf1\xf4\x9b\x19\xce\xbc\xf0\x02\x8a\xbc\xaf\xc2$\xfbZm\x89\xf7\xc1\xa9\xc0@\xfb?\x97\x16\xae I\xc6\x98\xab\x87\x95-\xde\xfd2&\xe2\xf8\x89.\x1b_\xa4:\xf68\x05\x18_\xadM\xd5\x85\xd0a_SW\xf18\xc9\xe8\xe2B!\x8e'


In [110]:
decrypted_corrupted_cipherdata = aes_cbc_cipher.decryptor().update(corrupted_cipherdata)
print(decrypted_corrupted_cipherdata)

b'To jest testowa wiadomosc. Powinna byc podzielon\x05G,\x0f\x92R\xeb\xef|\x1b\xdb4<\xd6\xbd\xc8w. Dalsza czesc wiadomosci. ABCDEFGHIJK\t\t\t\t\t\t\t\t\t'


### Błąd w szyfrogramie

In [51]:
num_of_blocks = len(im1mb_padded) // aes_block_size_in_bytes
margin = 1000
from_where_corrupt = randint(margin, num_of_blocks - margin)
corrupt_pos = from_where_corrupt * aes_block_size_in_bytes

first_half_1mb = aes_cbc_cipher.encryptor().update(im1mb_padded[0:corrupt_pos])

second_half_iv = corrupt_cipherdata(first_half_1mb[-aes_block_size_in_bytes:], 16)

aes_cbc_cipher2 = Cipher(AES(key), CBC(second_half_iv))
second_half_1mb = aes_cbc_cipher2.encryptor().update(im1mb_padded[corrupt_pos:])

cipherdata_1mb_corrupted = first_half_1mb + second_half_1mb

100%|██████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<?, ?it/s]


In [52]:
if not os.path.isdir('results/error_prop/cbc'):
    os.mkdir('results/error_prop/cbc')

In [53]:
decrypted_1b_corrupted = decrypt_image(cipherdata_1mb_corrupted, lambda data: aes_cbc_cipher.decryptor().update(data))
save_img('results/error_prop/cbc/1mb_corrupted.png', decrypted_1b_corrupted)

In [ ]:
show_image(filename='results/error_prop/ecb/1mb_corrupted.png', width=200)

### Błąd w szyfrogramie nr 2

In [120]:
cipherdata_1mb = aes_cbc_cipher.encryptor().update(im1mb_padded)
cipherdata_1mb_corrupted = corrupt_cipherdata(cipherdata_1mb, 10)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]


In [122]:
if not os.path.isdir('results/error_prop/cbc'):
    os.mkdir('results/error_prop/cbc')

In [123]:
decrypted_1b_corrupted = decrypt_image(cipherdata_1mb_corrupted, lambda data: aes_cbc_cipher.decryptor().update(data))
save_img('results/error_prop/cbc/1mb_corrupted2.png', decrypted_1b_corrupted)

In [ ]:
show_image(filename='results/error_prop/cbc/1mb_corrupted2.png', width=200)

## Moja implementacja CBC

In [55]:
def xor_bytes(bytes1, bytes2):
    return bytes([a ^ b for a, b in zip(bytes1, bytes2)])

In [56]:
def encrypt_cbc(aes_ecb_cipher, iv, data):
    num_of_blocks = len(data) // aes_block_size_in_bytes
    last_cipherdata = copy.copy(iv)
    encrypted_data = bytearray(num_of_blocks * aes_block_size_in_bytes)
    
    for block_id in tqdm(range(num_of_blocks)):
        start = block_id * aes_block_size_in_bytes
        stop = start + aes_block_size_in_bytes
        
        block_to_cipher = data[start:stop]
        data_to_cipher = xor_bytes(block_to_cipher, last_cipherdata)
        
        enc_block = aes_ecb_cipher.encryptor().update(data_to_cipher)
        encrypted_data[start:stop] = enc_block
        last_cipherdata = enc_block
        
    return bytes(encrypted_data)

In [57]:
def decrypt_cbc(aes_ecb_cipher, iv, data):
    num_of_blocks = len(data) // aes_block_size_in_bytes
    decrypted_data = bytearray(num_of_blocks * aes_block_size_in_bytes)
    
    for block_id in tqdm(range(num_of_blocks - 1, 0, -1)):
        curr_block_start = block_id * aes_block_size_in_bytes
        curr_block_stop = curr_block_start + aes_block_size_in_bytes
        
        prev_block_start = (block_id - 1) * aes_block_size_in_bytes
        prev_block_end = prev_block_start + aes_block_size_in_bytes
        
        curr_block = data[curr_block_start:curr_block_stop]
        prev_block = data[prev_block_start:prev_block_end]

        dec_curr_block = aes_ecb_cipher.decryptor().update(curr_block)
        decrypted_data[curr_block_start:curr_block_stop] = xor_bytes(dec_curr_block, prev_block)

    first_dec_block = aes_ecb_cipher.decryptor().update(data[0:aes_block_size_in_bytes])
    decrypted_data[0:aes_block_size_in_bytes] = xor_bytes(first_dec_block, iv)
    
    return bytes(decrypted_data)

In [58]:
msg = b'Test szyfrowania CBC. Dalsza tresc wiadomosci. ABCDEFG'
pkcs7_unpadder = padding.PKCS7(aes_block_size_in_bits).padder()
padded_msg = pkcs7_unpadder.update(msg) + pkcs7_unpadder.finalize()
print(len(padded_msg), len(padded_msg) * 8)
print(padded_msg)

64 512
b'Test szyfrowania CBC. Dalsza tresc wiadomosci. ABCDEFG\n\n\n\n\n\n\n\n\n\n'


In [59]:
res = encrypt_cbc(aes_ecb_cipher, iv, padded_msg)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]


In [60]:
dec_res = decrypt_cbc(aes_ecb_cipher, iv, res)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


In [61]:
dec_res

b'Test szyfrowania CBC. Dalsza tresc wiadomosci. ABCDEFG\n\n\n\n\n\n\n\n\n\n'

### Pomiar czasu szyfrowania mojej implementacji CBC

In [127]:
start = time.time()
cipherdata_1mb = encrypt_cbc(aes_ecb_cipher, iv, im1mb_padded)
stop = time.time()
enc_1mb_time = stop - start
print(f"Encryption time of my CBC on 1mb.png is {enc_1mb_time:.2f}s")

100%|████████████████████████████████████████████████████████████████████| 2160001/2160001 [00:16<00:00, 133608.11it/s]

Encryption time of my CBC on 1mb.png is 16.18s


In [63]:
encrypted_img_data = np.frombuffer(cipherdata_1mb, dtype=np.uint8)[:img_data.size].reshape(img_data.shape)

if not os.path.isdir("results/cbc"):
    os.mkdir("results/cbc")

encrypted_img = Image.fromarray(encrypted_img_data, 'RGBA')
encrypted_img.save('results/cbc/1mb.png')

In [ ]:
show_image(filename='results/cbc/1mb.png', width=200)

In [64]:
start = time.time()
cipherdata_5mb = encrypt_cbc(aes_ecb_cipher, iv, im5mb_padded)
stop = time.time()
enc_5mb_time = stop - start
print(f"Encryption time of my CBC on 5mb.png is {enc_5mb_time:.2f}s")

100%|██████████████████████████████████████████████████████████████████| 17113301/17113301 [02:07<00:00, 134375.75it/s]


Encryption time of my CBC on 5mb.png is 127.47s


In [65]:
start = time.time()
cipherdata_10mb = encrypt_cbc(aes_ecb_cipher, iv, im10mb_padded)
stop = time.time()
enc_10mb_time = stop - start
print(f"Encryption time of my CBC on 10mb.png is {enc_10mb_time:.2f}s")

100%|██████████████████████████████████████████████████████████████████| 42666001/42666001 [05:17<00:00, 134352.19it/s]


Encryption time of my CBC on 10mb.png is 317.86s


### Pomiar czasu deszyfrowania mojej implementacji CBC

In [66]:
start = time.time()
image_data_1mb = decrypt_cbc(aes_ecb_cipher, iv, cipherdata_1mb)
stop = time.time()
dec_1mb_time = stop - start
print(f"Decryption time of my CBC on 1mb.png is {dec_1mb_time:.2f}s")

100%|████████████████████████████████████████████████████████████████████| 2160000/2160000 [00:15<00:00, 142867.11it/s]

Decryption time of my CBC on 1mb.png is 15.13s


In [67]:
start = time.time()
image_data_5mb = decrypt_cbc(aes_ecb_cipher, iv, cipherdata_5mb)
stop = time.time()
dec_5mb_time = stop - start
print(f"Decryption time of my CBC on 5mb.png is {dec_5mb_time:.2f}s")

100%|██████████████████████████████████████████████████████████████████| 17113300/17113300 [02:01<00:00, 141220.71it/s]

Decryption time of my CBC on 5mb.png is 121.27s


In [68]:
start = time.time()
image_data_10mb = decrypt_cbc(aes_ecb_cipher, iv, cipherdata_10mb)
stop = time.time()
dec_10mb_time = stop - start
print(f"Decryption time of my CBC on 10mb.png is {dec_10mb_time:.2f}s")

100%|██████████████████████████████████████████████████████████████████| 42666000/42666000 [05:03<00:00, 140424.25it/s]


Decryption time of my CBC on 10mb.png is 304.09s


### Błąd w szyfrogramie używając moją implementację CBC

In [69]:
cipherdata_1mb = encrypt_cbc(aes_ecb_cipher, iv, im1mb_padded)
cipherdata_1mb_corrupted = corrupt_cipherdata(cipherdata_1mb, 10)
assert cipherdata_1mb != cipherdata_1mb_corrupted

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]


In [70]:
if not os.path.isdir('results/error_prop/moj_cbc'):
    os.mkdir('results/error_prop/moj_cbc')

In [71]:
decrypted_1b_corrupted = decrypt_image(cipherdata_1mb_corrupted, lambda data: decrypt_cbc(aes_ecb_cipher, iv, data))
save_img('results/error_prop/moj_cbc/1mb_corrupted.png', decrypted_1b_corrupted)

100%|████████████████████████████████████████████████████████████████████| 2160000/2160000 [00:15<00:00, 141802.84it/s]


In [ ]:
show_image(filename='results/error_prop/moj_cbc/1mb_corrupted.png', width=200)

## CTR

In [73]:
iv = os.urandom(aes_block_size_in_bytes)
aes_ctr_cipher = Cipher(AES(key), CTR(iv))

### Pomiar czasu szyfrowania ECB

In [74]:
cipherdata_1mb, cipherdata_5mb, cipherdata_10mb = measure_enc_time(lambda data: aes_ctr_cipher.encryptor().update(data))

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.15it/s]

For 1mb.png, mean = 0.0166s, var = 0.0000s, median = 0.0165s
For 5mb.png, mean = 0.1279s, var = 0.0000s, median = 0.1273s
For 10mb.png, mean = 0.3189s, var = 0.0000s, median = 0.3196s


### Pomiar czasu deszyfrowania ECB

In [75]:
measure_dec_time(lambda data: aes_ctr_cipher.decryptor().update(data), cipherdata_1mb, cipherdata_5mb, cipherdata_10mb)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:23<00:00,  2.11it/s]

For 1mb.png, mean = 0.0176s, var = 0.0000s, median = 0.0165s
For 5mb.png, mean = 0.1314s, var = 0.0001s, median = 0.1300s
For 10mb.png, mean = 0.3249s, var = 0.0002s, median = 0.3238s


### Błąd w szyfrogramie

In [115]:
cipherdata_1mb = aes_ctr_cipher.encryptor().update(im1mb_padded)
cipherdata_1mb_corrupted = corrupt_cipherdata(cipherdata_1mb, 10)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]


In [116]:
if not os.path.isdir('results/error_prop/ctr'):
    os.mkdir('results/error_prop/ctr')

In [117]:
decrypted_1b_corrupted = decrypt_image(cipherdata_1mb_corrupted, lambda data: aes_ctr_cipher.decryptor().update(data))
save_img('results/error_prop/ctr/1mb_corrupted.png', decrypted_1b_corrupted)

In [ ]:
show_image(filename='results/error_prop/ctr/1mb_corrupted.png', width=200)

### Zapisanie zaszyfrowanego tuxa

In [119]:
encrypted_img_data = np.frombuffer(cipherdata_1mb, dtype=np.uint8)[:img_data.size].reshape(img_data.shape)
if not os.path.isdir("results/ctr"):
    os.mkdir("results/ctr")
encrypted_img = Image.fromarray(encrypted_img_data, 'RGBA')
encrypted_img.save('results/ctr/1mb.png')

In [ ]:
show_image(filename='results/ctr/1mb.png', width=200)

> Czy błąd w szyfrogramie będzie skutkował niemożnością odczytania po deszyfrowaniucałej wiadomości, fragmentu?

Odp: **Nie**. I to w żadnym trybie. W CBC można zakładać, że tak będzie, bo kolejne bloki są zależne od poprzednich, ale ani uszkodzenie bitów przed deszyfrowaniem i uszkodzenie bitów podczas szyfrowania symulując transmisje strumieniową nie powodują większych problemów. Maksymalnie problem ogranicza się do jednego bloku. Jest tak, bo kolejne bloki są już dobrze zaszyfrowane i dobrze odszyfrowane, bo sąsiednie bloki są poprawne.